# Analyze Chemical Space

In [3]:
import importlib
import os
import sys
import statistics

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import torch

import helper
from modelforge.dataset.dataset import initialize_datamodule
from modelforge.dataset.utils import RandomRecordSplittingStrategy, SplittingStrategy

sns.set_theme()
sns.set_context("notebook")

#importlib.reload(helper)

In [8]:
# initialize datasets as in training
dm = initialize_datamodule(
    dataset_name="spice2",
    version_select="full_dataset_HCNOF_v1.1",
    local_cache_dir="../cache",
    dataset_cache_dir="./dataset_cache",
    properties_of_interest=["atomic_numbers", "positions", "dft_total_energy", "dft_total_force", "total_charge", "scf_dipole"],
    properties_assignment={
        "atomic_numbers": "atomic_numbers",
        "positions": "positions",
        "E": "dft_total_energy",
        "F": "dft_total_force",
        "total_charge": "total_charge",
        "dipole_moment": "scf_dipole"
    }
)

2025-10-01 19:02:33.805 | DEBUG    | modelforge.dataset.dataset:__init__:327 - Loading config data from /Users/syan/workdir/modelforge/modelforge/dataset/yaml_files/spice2.yaml
2025-10-01 19:02:33.830 | INFO     | modelforge.dataset.dataset:__init__:367 - Using dataset version full_dataset_HCNOF_v1.1
2025-10-01 19:02:33.832 | WARNING  | modelforge.dataset.dataset:_metadata_validation:680 - Checksum for hdf5 file used to generate npz file does not match current file in dataloader.
2025-10-01 19:02:33.832 | DEBUG    | modelforge.dataset.dataset:_file_validation:721 - File ./dataset_cache/spice_2_dataset_v1.1_HCNOF.hdf5 does not exist.
2025-10-01 19:02:33.833 | DEBUG    | modelforge.dataset.dataset:_acquire_dataset:461 - hdf5 file spice_2_dataset_v1.1_HCNOF.hdf5 not found.
2025-10-01 19:02:33.921 | DEBUG    | modelforge.utils.remote:download_from_url:195 - Downloading datafile from https://zenodo.org/records/15579090/files/spice_2_dataset_v1.1_HCNOF.hdf5.gz to ./dataset_cache/spice_2_data

Validating file ./dataset_cache/spice_2_dataset_v1.1_HCNOF.hdf5


downloading: 100%|##########| 22875180/22875180 [16:54<00:00, 22548.19it/s]
2025-10-01 19:20:22.700 | DEBUG    | modelforge.dataset.dataset:_from_hdf5:795 - Reading data from ./dataset_cache/spice_2_dataset_v1.1_HCNOF.hdf5
2025-10-01 19:20:22.701 | DEBUG    | modelforge.dataset.dataset:_from_hdf5:796 - element filter: None
2025-10-01 19:20:24.568 | DEBUG    | modelforge.dataset.dataset:_from_hdf5:828 - Properties of Interest: ['atomic_numbers', 'positions', 'dft_total_energy', 'dft_total_force', 'total_charge', 'scf_dipole']
2025-10-01 19:20:24.569 | DEBUG    | modelforge.dataset.dataset:_from_hdf5:834 - n_entries: 57037
100%|██████████| 57037/57037 [02:01<00:00, 469.61it/s]
2025-10-01 19:22:26.319 | DEBUG    | modelforge.dataset.dataset:_to_file_cache:1046 - Writing npz file to ../cache/spice2.npz
2025-10-01 19:22:26.560 | DEBUG    | modelforge.dataset.dataset:_from_file_cache:1007 - Loading processed data from ../cache/spice2.npz generated on 2025-10-01 19:22:26.552259
2025-10-01 19:

Validating file ../cache/spice2.npz


Calculating pairlist for dataset: 100%|██████████| 4641/4641 [00:20<00:00, 231.00it/s]
2025-10-01 19:22:54.868 | INFO     | modelforge.dataset.utils:calculate_mean_and_variance:167 - Calculating mean and variance of atomic energies
100%|██████████| 1813/1813 [00:09<00:00, 186.34it/s]
2025-10-01 19:23:04.602 | INFO     | modelforge.dataset.utils:calculate_mean_and_variance:179 - Mean and standard deviation of the dataset:{'per_atom_energy_mean': tensor(-430.6900, dtype=torch.float64), 'per_atom_energy_stddev': tensor(42.4628, dtype=torch.float64)}
2025-10-01 19:23:04.603 | INFO     | modelforge.dataset.dataset:prepare_data:1284 - {'atomic_self_energies': {'B': <Quantity(-24.6715205, 'hartree')>, 'Br': <Quantity(-2574.11672, 'hartree')>, 'C': <Quantity(-37.8726451, 'hartree')>, 'Ca': <Quantity(-676.952847, 'hartree')>, 'Cl': <Quantity(-460.198876, 'hartree')>, 'F': <Quantity(-99.7861162, 'hartree')>, 'H': <Quantity(-0.49876051, 'hartree')>, 'I': <Quantity(-297.762289, 'hartree')>, 'K': <

## Plotting

In [14]:
vars(dm)

{'_log_hyperparams': False,
 'prepare_data_per_node': True,
 'allow_zero_length_dataloader_with_multiple_devices': False,
 'trainer': None,
 'name': 'spice2',
 'batch_size': 64,
 'splitting_strategy': <modelforge.dataset.utils.FirstComeFirstServeSplittingStrategy at 0x341e35850>,
 'remove_self_energies': True,
 'shift_center_of_mass_to_origin': False,
 'dict_atomic_self_energies': None,
 'regression_ase': False,
 'force_download': False,
 'regenerate_processed_dataset': False,
 'version_select': 'full_dataset_HCNOF_v1.1',
 'train_dataset': <torch.utils.data.dataset.Subset at 0x343259f10>,
 'val_dataset': <torch.utils.data.dataset.Subset at 0x3d0a1f5f0>,
 'test_dataset': <torch.utils.data.dataset.Subset at 0x49009fec0>,
 'properties_of_interest': ['atomic_numbers',
  'positions',
  'dft_total_energy',
  'dft_total_force',
  'total_charge',
  'scf_dipole'],
 'properties_assignment': {'atomic_numbers': 'atomic_numbers',
  'positions': 'positions',
  'E': 'dft_total_energy',
  'F': 'dft_to

In [23]:
vars(dm.torch_dataset)

{'preloaded': False,
 'properties_of_interest': {'atomic_numbers': tensor([8, 8, 6,  ..., 1, 1, 1], dtype=torch.int32),
  'positions': tensor([[ 0.4113, -0.2958,  0.0557],
          [ 0.7071,  0.1196,  0.0680],
          [ 0.4835, -0.2891,  0.1545],
          ...,
          [ 0.4428, -0.2938, -0.1836],
          [-0.1426, -0.5384,  0.1532],
          [-0.2655, -0.4963,  0.2526]]),
  'E': tensor([[-27507.4269],
          [-27497.7733],
          [-27488.4544],
          ...,
          [-29707.2108],
          [-29707.5980],
          [-29613.8078]], dtype=torch.float64),
  'total_charge': tensor([[0],
          [0],
          [0],
          ...,
          [0],
          [0],
          [0]], dtype=torch.int32),
  'F': tensor([[  249.9994,   496.8468,    97.5443],
          [ -352.4402,  -201.2924,   251.3811],
          [-1088.0614, -1128.3499,  -807.4728],
          ...,
          [ 1188.5764,  1633.7742,  1061.7694],
          [ -236.1393,   380.6429,   776.6714],
          [  588.8116

In [35]:
dm.torch_dataset.properties_of_interest["total_charge"].numpy().reshape(-1).mean()

0.0